In [0]:
!pip install --upgrade -q gspread
!pip install lxml

In [0]:
from lxml import html
import requests
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import time

# SDK autorize
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Open sheet and read some data.
worksheet = gc.open('LinkedinDATA').sheet1

# Get all "domain.com" from Sheet1 
sites = worksheet.get_all_values()

# Open Sheet 2 and add some data.
worksheet = gc.open('LinkedinDATA').get_worksheet(1)

rowC = 1
rowP = 1
column = 1

for site in sites:
  i = 0
  n = 0
  pageCompany = requests.get('https://search.yahoo.com/search?p=site:linkedin.com/company/ "' + site[0] + '"')
  pagePersonal = requests.get('https://search.yahoo.com/search?p=site:linkedin.com/in/ "' + site[0] + '"')
  treeCompany = html.fromstring(pageCompany.content)
  treePersonal = html.fromstring(pagePersonal.content)
  company = treeCompany.xpath('//li/div/div/h3[@class="title"]/a/@href')
  personal = treePersonal.xpath('//li/div/div/h3[@class="title"]/a/@href')
  print ('Link for: ',site)
  row = max(rowC, rowP) + 1
  worksheet.update_cell(row, 1, site[0])
  time.sleep(2)
  while i < len(company) and i < 3:
    print (company[i])
    rowC += 1
    worksheet.update_cell(rowC, 2, company[i])
    i += 1
  time.sleep(2)
  if len(company) == 0:
    worksheet.update_cell(row, 2, 'No data')
    row += 1
    rowC += 1
  while n < len(personal) and n < 3:
    print (personal[n])
    rowP += 1
    worksheet.update_cell(rowP, 3, personal[n])
    n += 1
  time.sleep(2)
  if len(personal) == 0:
    worksheet.update_cell(row, 3, 'No data')
    row += 1
    rowC += 1
  rowC = max(rowC,rowP)
  rowP = max(rowC,rowP)
  time.sleep(5)